# 1. Чтение текста из файла в строку

In [1]:
def read_from_file(filename):
    string = ''
    with open(filename, "r", encoding='utf-8') as file:
        for line in file:
            string += line
    return string

# 2. Запись raw формата изображения в файл

In [2]:
def raw_to_file(array, filename):
    with open(filename, "wb") as file:
        for item in array:
            file.write(item.to_bytes(1, byteorder='big', signed=False))
    return

# 3. Чтение raw формата изображения из файла

In [3]:
def raw_from_file(filename):
    array = bytearray()
    with open(filename, "rb") as file:
        for item in file:
            array += item
    return array

# 4. RLE кодирование

In [41]:
def rle_encode(data):
    if type(data) == str: data += '$'
    else: data.append(0)
    encoded_data = []
    count = 1
    for i in range(1, len(data)):
        if data[i] == data[i-1]:
            count += 1
        else:
            if count > 1: encoded_data.append((data[i-1], count))
            else: encoded_data.append(data[i-1])
            count = 1
    single_count = 0
    index = -1
    labled_encoded_data = encoded_data.copy()
    k = 0
    for i in range(len(encoded_data)):
        if not isinstance(encoded_data[i], tuple):
            if single_count == 0: index = i
            single_count += 1
            if i == len(encoded_data)-1:
                if single_count != 0:
                    labled_encoded_data.insert(index+k, -single_count)
                break
        else:
            if single_count != 0:
                labled_encoded_data.insert(index+k, -single_count)
                k += 1
            single_count = 0   
    
    return labled_encoded_data

# 5. Запись закодированной RLE последовательности в файл

In [5]:
def rle_to_file(encoded_data, filename, symbol_bytes, text_or_image):
    if text_or_image == 'text':
        with open(filename, 'wb') as file:
            for item in encoded_data:
                if isinstance(item, tuple):
                    file.write(item[1].to_bytes(2, byteorder='big', signed=True))
                    file.write(ord(item[0]).to_bytes(symbol_bytes, byteorder='big', signed=False))
                else:
                    if type(item) == int and item < 0:
                        file.write(item.to_bytes(2, byteorder='big', signed=True))
                    else:
                        try:
                            file.write(ord(item).to_bytes(symbol_bytes, byteorder='big', signed=False))
                        except:
                            print(item)
    elif text_or_image == 'image':
        with open(filename, 'wb') as file:
            for item in encoded_data:
                if isinstance(item, tuple):
                    file.write(item[1].to_bytes(2, byteorder='big', signed=True))
                    file.write(item[0].to_bytes(symbol_bytes, byteorder='big', signed=False))
                else:
                    if item < 0:
                        file.write(item.to_bytes(2, byteorder='big', signed=True))
                    else:
                        file.write(item.to_bytes(symbol_bytes, byteorder='big', signed=False))
    return

# 6. Декодирование RLE из файла

In [ ]:
def decode_from_file(filename, symbol_bytes, text_or_image):
    if text_or_image == 'text':
        decoded_string = ''
        with open(filename, 'rb') as file:
            while True:
                read_bytes = file.read(2)
                if not read_bytes:
                    break
                int_val = int.from_bytes(read_bytes, "big", signed=True)
                if int_val > 0:
                    letter = chr(int.from_bytes(file.read(symbol_bytes), "big", signed=False))
                    for _ in range(int_val):
                        decoded_string += letter
                else:
                    for i in range(-int_val):
                        decoded_string += chr(int.from_bytes(file.read(symbol_bytes), "big", signed=False))
        return decoded_string
    elif text_or_image == 'image':
        decoded_list = []
        with open(filename, 'rb') as file:
            while True:
                read_bytes = file.read(2)
                if not read_bytes:
                    break
                int_val = int.from_bytes(read_bytes, "big", signed=True)
                if int_val > 0:
                    color = int.from_bytes(file.read(symbol_bytes), "big", signed=False)
                    for _ in range(int_val):
                        decoded_list.append(color)
                else:
                    for i in range(-int_val):
                        decoded_list.append(int.from_bytes(file.read(symbol_bytes), "big", signed=False))
        return decoded_list
    return

# 7. Приведение изображения к заданному разрешению

In [10]:
from PIL import Image
import numpy as np

def resize_image(image_path, target_resolution):
    original_image = Image.open(image_path)
    resized_image = original_image.resize(target_resolution, Image.BILINEAR)
    return resized_image

# 8. Приведение ЧБ изображения к raw формату

In [7]:
def image_to_raw_white_and_black(image_path):
    # Открываем изображение с помощью PIL
    img = Image.open(image_path)
    # Преобразуем изображение в массив numpy
    img_array = np.array(img)
    # Приводим значения к диапазону от 0 до 255
    img_array = img_array.astype(np.uint8)
    # Получаем форму изображения
    height, width, _ = img_array.shape
    # Создаем пустой массив для raw данных
    raw_data = bytearray()
    # Добавляем значения цветности в raw данные
    for i in range(height):
        for j in range(width):
            img_array[i, j, 0] = 255 if img_array[i, j, 0] > 132 else 0
            raw_data.append(img_array[i, j, 0])
    return raw_data, height, width

# 9. Приведение серого изображения к raw формату

In [8]:
def image_to_raw_gray(image_path):
    # Открываем изображение с помощью PIL
    img = Image.open(image_path)
    # Преобразуем изображение в массив numpy
    img_array = np.array(img)
    # Приводим значения к диапазону от 0 до 255
    img_array = img_array.astype(np.uint8)
    # Получаем форму изображения
    height, width, _ = img_array.shape
    # Создаем пустой массив для raw данных
    raw_data = bytearray()
    # Добавляем значения цветности в raw данные
    for i in range(height):
        for j in range(width):
            raw_data.append(img_array[i, j, 0])
    return raw_data, height, width

# 10. Приведение RGB изображения к raw формату

In [9]:
def image_to_raw(image_path):
    # Открываем изображение с помощью PIL
    img = Image.open(image_path)
    # Преобразуем изображение в массив numpy
    img_array = np.array(img)
    # Приводим значения к диапазону от 0 до 255
    img_array = img_array.astype(np.uint8)
    # Получаем форму изображения
    height, width, channels = img_array.shape
    # Создаем пустой массив для raw данных
    raw_data = bytearray()
    # Добавляем значения цветности в raw данные
    for i in range(height):
        for j in range(width):
            for k in range(channels):
                raw_data.append(img_array[i, j, k])
    return raw_data, height, width, channels

# 11. Рассчёт энтропии

In [11]:
import math

def entropy(probabilities):
    entropy_value = 0.0
    for probability in probabilities:
        if probability != 0:
            entropy_value -= probability * math.log2(probability)
    return entropy_value

# Пример использования
probabilities = [0.1, 0.2, 0.3, 0.4]  # Пример массива вероятностей
entropy_value = entropy(probabilities)
print("Entropy:", entropy_value)

Entropy: 1.8464393446710154


# 12. Построение дерева Хаффмана

In [12]:
import heapq
from collections import Counter, defaultdict

class HuffmanNode:
    def __init__(self, value, frequency):
        self.value = value
        self.frequency = frequency
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequency < other.frequency

def build_huffman_tree(text):
    frequency_counter = Counter(text)
    priority_queue = [HuffmanNode(symbol, freq) for symbol, freq in frequency_counter.items()]
    heapq.heapify(priority_queue)

    while len(priority_queue) > 1:
        left_child = heapq.heappop(priority_queue)
        right_child = heapq.heappop(priority_queue)

        merged_node = HuffmanNode(None, left_child.frequency + right_child.frequency)
        merged_node.left = left_child
        merged_node.right = right_child

        heapq.heappush(priority_queue, merged_node)

    return priority_queue[0]

# 13. Построение кодов Хаффмана

In [13]:
def build_huffman_codes(node, prefix="", codes={}):
    if node:
        if node.value is not None:
            codes[node.value] = prefix
        build_huffman_codes(node.left, prefix + "0", codes)
        build_huffman_codes(node.right, prefix + "1", codes)
    return codes

# 14. Кодирование текста алгоритмом сжатия Хаффмана

In [14]:
def huffman_encode(text, codes):
    encoded_text = ""
    for symbol in text:
        encoded_text += codes[symbol]
    return encoded_text

# 15. Декодирование строки из кодов Хаффмана

In [15]:
def huffman_decode(encoded_text, tree):
    decoded_text = ""
    current_node = tree
    for bit in encoded_text:
        if bit == '0':
            current_node = current_node.left
        else:
            current_node = current_node.right

        if current_node.value is not None:
            decoded_text += current_node.value
            current_node = tree
    return decoded_text

# 16. Набор канонических кодов

In [16]:
def canonical_huffman_codes(huffman_codes):
    sorted_codes = sorted((len(code), code) for code in huffman_codes)
    canonical_codes = [''] * len(huffman_codes)
    length = 0
    for i, (l, code) in enumerate(sorted_codes):
        if l > length:
            length = l
        canonical_codes[i] = code + '0' * (length - l)
    return canonical_codes

# 17. Построение канонических кодов по массиву длин

In [18]:
# Функция для инкрементирования бинарной строки
def increment_binary_string(binary_string):
    binary_list = list(binary_string)
    for i in range(len(binary_list) - 1, -1, -1):
        if binary_list[i] == '0':
            binary_list[i] = '1'
            break
        else:
            binary_list[i] = '0'
    return ''.join(binary_list)

def canonical_huffman_codes_from_lengths(code_lengths):
    # Создаем список пар (длина, индекс)
    sorted_lengths = sorted((length, index) for index, length in enumerate(code_lengths))

    # Инициализируем массив канонических кодов
    canonical_codes = [''] * len(code_lengths)

    # Предыдущий код
    previous_code = ""

    # Текущая длина кода
    current_length = 0

    for length, index in sorted_lengths:
        # Если длина кода увеличивается, добавляем '0' к предыдущему коду
        if length > current_length:
            previous_code += '0' * (length - current_length)
            current_length = length

        # Строим канонический код, добавляя по одному биту
        canonical_codes[index] = previous_code
        previous_code = increment_binary_string(previous_code)

    return canonical_codes

# 18. Запись последовательности из кодов Хаффмана в файл

In [19]:
def huffman_to_file(filename):
    with open(filename, "wb") as file:
        for sub in range(0, len(encoded_text), 8):
            byte = int(encoded_text[sub:sub+8],  2)
            file.write(byte.to_bytes(1, byteorder='big'))
        return

# 19. Чтение последовательности из кодов Хаффмана из файла

In [20]:
def huffman_from_file(filename):
    string = ''
    with open(filename, "rb") as file:
        while True:
            byte = file.read(1)
            if not byte:
                break
            code = bin(int.from_bytes(byte, byteorder='big'))[2:]
            if len(code) < 8: code = '0' * (8 - len(code)) + code
            string += code
        return string

# 20. Арифметическое кодирование

In [21]:
def arithmetic_coding_encode(input_string, frequency_dict, step=4):
    # Инициализируем начальные значения
    low = 0.0
    high = 1.0
    values_list = []

    i = 0
    # Проходимся по каждому символу в строке
    for symbol in input_string:

        # Обновляем границы интервала
        symbol_index = list(frequency_dict.keys()).index(symbol)
        new_low = low + (high - low) * (list(frequency_dict.values())[symbol_index - 1] if symbol_index != 0 else 0)
        new_high = low + (high - low) * list(frequency_dict.values())[symbol_index]

        # Обновляем нижнюю границу
        low = new_low

        # Обновляем верхнюю границу
        high = new_high

        i += 1
        if i == step or step * len(values_list) + i == len(input_string):
            values_list.append((low + high) / 2)
            low = 0.0
            high = 1.0
            i = 0

    return values_list

# 21. Декодирование арифметического кодирования

In [22]:
def arithmetic_coding_decode(encoded_values, frequency_dict, input_string_length, step=4):
    # Восстанавливаем исходную строку
    decoded_string = ""

    i = 0
    k = 0
    for _ in range(input_string_length):
        # Ищем символ в частотном словаре
        for symbol, symbol_high in frequency_dict.items():

            symbol_index = list(frequency_dict.keys()).index(symbol)
            symbol_low = list(frequency_dict.values())[symbol_index - 1] if symbol_index != 0 else 0
            # Если значение в пределах интервала для текущего символа
            if symbol_low <= encoded_values[k]+pow(10, -10) < symbol_high:
                decoded_string += symbol

                i += 1
                if i == step:
                    k += 1
                    i = 0
                else:
                    encoded_values[k] = (encoded_values[k] - symbol_low) / (symbol_high - symbol_low)

                break

    return decoded_string

# 22. Построение словаря с границами

In [23]:
from collections import Counter

def build_frequency_dict(text):
    frequency_dict = Counter(text)

    num_frequency_dict = {}
    point = 1 / sum(frequency_dict.values())
    num = 0
    for key in frequency_dict:
        num += point * frequency_dict[key]
        num_frequency_dict[key] = num

    return num_frequency_dict

# 23. Запись арифметического кодирования в файл

In [24]:
import struct

def arithmetic_to_file(filename, encoded_text, dictionary, text_len):
    with open(filename, 'wb') as file:
        file.write(text_len.to_bytes(6, byteorder='big'))
        for item in encoded_text:
            file.write(bytearray(struct.pack("f", item)))
        for key in dictionary:
            file.write(ord(key).to_bytes(2, byteorder='big'))
            file.write(bytearray(struct.pack("f", dictionary[key])))
    return

# 24. Чтение арифметического кодирования из файла

In [25]:
def arithmetic_from_file(filename, step=4):
    with open(filename, 'rb') as file:
        text_len = int.from_bytes(file.read(6), byteorder='big')
        encoded_text = []
        for i in range(math.ceil(text_len / step)):
            encoded_text.append(struct.unpack("f", file.read(4))[0])
        
        dictionary = {}
        while True:
            tmp = file.read(2)
            if not tmp:
                break
            letter = chr(int.from_bytes(tmp, byteorder='big'))
            count = struct.unpack("f", file.read(4))[0]
            dictionary[letter] = count
    
    return encoded_text, dictionary, text_len

# 25. Наивная реализация BWT

In [27]:
def burrows_wheeler_transform(text):
    rotations = [text[i:] + text[:i] for i in range(len(text))]
    sorted_rotations = sorted(rotations)
    bwt = ''.join(rotation[-1] for rotation in sorted_rotations)
    idx = sorted_rotations.index(text)
    return bwt, idx

# 26. Обратное преобразование BWT

In [28]:
def inverse_burrows_wheeler_transform(bwt, idx):
    table = [''] * len(bwt)
    for i in range(len(bwt)):
        table = sorted(bwt[i] + table[i] for i in range(len(bwt)))
    text = table[idx]
    return text

# 27. Нахождение средней длинны последовательностей с повторяющимися символами

In [29]:
def average_repeated_sequence_length(s):
    total_length = 0
    repeated_sequences_count = 0

    i = 0
    while i < len(s):
        current_char = s[i]
        sequence_length = 1

        # Count the length of the repeated sequence
        while i + 1 < len(s) and s[i + 1] == current_char:
            sequence_length += 1
            i += 1

        if sequence_length > 1:
            total_length += sequence_length
            repeated_sequences_count += 1

        i += 1

    if repeated_sequences_count == 0:
        return 0

    average_length = total_length / repeated_sequences_count
    result = (total_length - 2 * repeated_sequences_count) / len(s)

    return average_length, result

# 28. Наивная реализация построения суффиксного массива

In [30]:
def naive_suffix_array(text):
    suffixes = [(text[i:], i) for i in range(len(text))]
    suffixes.sort(key=lambda x: x[0])
    return [suffix[1] for suffix in suffixes]

# 29. Получение последнего столбца BWT с помощью суффиксного массива

In [31]:
def bwt_last_column(text, suffix_array):
    bwt = ''.join(text[i - 1] for i in suffix_array)
    return ''.join(bwt)

# 30. Определение типа суффикса

In [32]:
def suffix_types(text):
    n = len(text)
    types = [''] * n
    
    # Определяем тип суффикса для '$' как 'S'
    types[n - 1] = 'S'
    
    # Проходим по символам строки справа налево
    for i in range(n - 2, -1, -1):
        # Если текущий символ меньше следующего, то суффикс типа 'S'
        if text[i] < text[i + 1]:
            types[i] = 'S'
        # Если символы равыны, то суффикс типа предыдущего
        elif text[i] == text[i + 1]:
            types[i] = types[i + 1]
        # Иначе суффикс типа 'L'
        else:
            types[i] = 'L'
    
    return ''.join(types)

# 31. Эффективная реализация обратного преобразования BWT

In [36]:
# Python program for the above approach

import string

# Structure to store info of a node of linked list
class Node:
	def __init__(self, data):
		self.data = data
		self.next = None

# Does insertion at end in the linked list
def addAtLast(head, nn):
	if head is None:
		head = nn
		return head
	temp = head
	while temp.next is not None:
		temp = temp.next
	temp.next = nn
	return head

# Computes l_shift[]
def computeLShift(head, index, l_shift):
	l_shift[index] = head.data
	head = head.next

# Compares the characters of bwt_arr[] and sorts them alphabetically
def cmpfunc(a, b):
	return ord(a) - ord(b)

def invert(bwt_arr):
	len_bwt = len(bwt_arr)
	sorted_bwt = sorted(bwt_arr)
	l_shift = [0] * len_bwt

	# Index at which original string appears
	# in the sorted rotations list
	x = 4

	# Array of lists to compute l_shift
	arr = [[] for i in range(256)]

	# Adds each character of bwt_arr to a linked list
	# and appends to it the new node whose data part
	# contains index at which character occurs in bwt_arr
	for i in range(len_bwt):
		arr[ord(bwt_arr[i])].append(i)

	# Adds each character of sorted_bwt to a linked list
	# and finds l_shift
	for i in range(len_bwt):
		l_shift[i] = arr[ord(sorted_bwt[i])].pop(0)

	# Decodes the bwt
	decoded = [''] * len_bwt
	for i in range(len_bwt):
		x = l_shift[x]
		decoded[len_bwt-1-i] = bwt_arr[x]
	decoded_str = ''.join(decoded)

	return decoded_str[::-1]

bwt_arr = "annb$aa"
invert(bwt_arr)

'banana$'

# 32. Чтение из файла вместе с преобразованием BWT частей текста

In [40]:
def read_from_file_with_bwt(filename):
    string = ''
    with open(filename, "r", encoding='utf-8') as file:
        i = 1
        tmp_string = ''
        for line in file:
            tmp_string += line
            if i % 300 == 0:
                suffix_array = naive_suffix_array(tmp_string)
                string += bwt_last_column(tmp_string, suffix_array) + '$'
                i = 1
                tmp_string = ''
            i += 1
        suffix_array = naive_suffix_array(tmp_string)
        string += bwt_last_column(tmp_string, suffix_array) + '$'
    return string

# 33. Декодирование BWT по блокам

In [37]:
def bwt_bloks_decoding(string):
    tmp_string = ''
    decoded_string = ''
    for sub in string:
        if sub != '$':
            tmp_string += sub
        else:
            inverted_string = invert(tmp_string)
            if inverted_string != None:
                decoded_string += invert(tmp_string)
            tmp_string = ''
    return decoded_string

# 34. MTF

In [38]:
def move_to_front_encode(s):
    alphabet = list(range(256))  # Создаем алфавит из всех возможных символов (ASCII)
    encoded = []

    for char in s:
        index = alphabet.index(ord(char))  # Находим индекс символа в алфавите
        encoded.append(index)  # Добавляем индекс в закодированную строку
        # Перемещаем символ в начало алфавита
        alphabet.insert(0, alphabet.pop(index))

    return encoded

# 35. Обртаное преобразование MTF

In [39]:
def move_to_front_decode(encoded):
    alphabet = list(range(256))  # Создаем алфавит из всех возможных символов (ASCII)
    decoded = []

    for index in encoded:
        char = chr(alphabet[index])  # Находим символ по индексу в алфавите
        decoded.append(char)  # Добавляем символ в декодированную строку
        # Перемещаем символ в начало алфавита
        alphabet.insert(0, alphabet.pop(index))

    return ''.join(decoded)